In [ ]:

!pip install selenium
!pip install webdriver_manager
import sys
sys.path.insert(0,r"c/Users/noblenelson/chromedriver 4")
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')

wd = webdriver.Chrome(ChromeDriverManager().install())

In [ ]:
wd.get("https://www.linkedin.com/")
print(wd.title)


In [ ]:
#username = input('Enter your LinkedIn username: ')
#password = input ('Enter your LinkedIn password: ')
username_field = wd.find_element_by_id('session_key')
username_field.send_keys("noblenelson26@gmail.com")
password_field = wd.find_element_by_id('session_password')
password_field.send_keys("@Bcd3110")
submit_button = wd.find_element_by_class_name('sign-in-form__submit-button')
submit_button.click()

In [ ]:
wd.implicitly_wait(3)
try:
  search_field = wd.find_element_by_css_selector(".search-global-typeahead__collapsed-search-button-icon")
except:
  if wd.title == 'Security Verification | LinkedIn':
    # LinkedIn is asking for a verification code. Check your email and input the code here
    code = input('Enter the verification code: ')
    verification_input = wd.find_element_by_css_selector('.input_verification_pin')
    verification_input.clear()
    verification_input.send_keys(code)
    submit_button = wd.find_element_by_id('.email-pin-submit-button')
    submit_button.click()
    # Implicit wait already set to 10 seconds
    search_field = wd.find_element_by_css_selector(".search-global-typeahead__collapsed-search-button-icon")

In [ ]:

def extract_jobs_on_page(result_list):
    
    for item in result_list:
        job = {}
        
        try:
            company = item.find_element_by_css_selector('.job-card-container__company-name').text # class changed
        except:
            company = ""
        try:
            title = item.find_element_by_css_selector('.job-card-list__title').text # class changed
        except:
            title = ""
        try:
            location = item.find_element_by_css_selector('.job-card-container__metadata-item').text # class changed
        except:
            location = ""
        try:
            posted = item.find_element_by_css_selector('.job-card-container__listed-time').text # class changed
        except:
            posted = ""
        try:
            url = item.find_element_by_css_selector('.job-card-container__link').get_attribute("href") # class changed
        except:
            url = ""
        job = {
            "company": company,
            "title": title,
            "location": location,
            "posted": posted,
            "href": url
        }
        return job

In [ ]:
from selenium import webdriver
search_url = 'https://www.linkedin.com/jobs/search/?geoId=90009551&keywords=data%20analyst&location=Greater%20Toronto%20Area%2C%20Canada'
wd.get(search_url)
time.sleep(10)   # put time sleep
result_list = wd.find_elements_by_css_selector(".jobs-search-results__list-item")  # element to elements and class change
extract_jobs_on_page(result_list)#print(jobs)

In [ ]:
search_url = 'https://www.linkedin.com/jobs/search/?geoId=90009551&keywords=data%20analyst&location=Greater%20Toronto%20Area%2C%20Canada'
jobs = []
for i in range(87,108):
    wd.get(search_url + '&start=' + str(i*1))
    time.sleep(10)   # put time sleep
    result_list = wd.find_elements_by_css_selector('.jobs-search-results__list.list-style-none')
    job = extract_jobs_on_page(result_list)
    jobs.append(job)  
    


In [ ]:
full_jobs = []
for job in jobs:
    if job.get("href"):
        wd.get(job['href'])
        description = wd.find_element_by_css_selector('.jobs-description-content__text')
        job['description'] = description.text
        skills = []
        skill_list = wd.find_elements_by_css_selector('.jobs-ppc-criteria__list--skills li .jobs-ppc-criteria__value')
        for skill in skill_list:
            if(skill.text.strip() != ''):
                skills.append(skill.text.strip())
        job['skills'] = skills
        industry_list = wd.find_elements_by_css_selector('.js-formatted-industries-list li')
        industries = []
        for industry in industry_list:
            industry_name = industry.get_attribute('innerHTML')
            if (industry_name.strip() != ''):
                industries.append(industry_name.strip())
        job['industry'] = industries
        full_jobs.append(job)
print(full_jobs)


In [ ]:

lines = []
for record in range(1,20):
    line = []
    line.append('"' + full_jobs[record]['company'].replace('"', '\"') + '"')
    line.append('"' + full_jobs[record]['title'].replace('"', '\"') + '"')
    line.append('"' + full_jobs[record]['location'].replace('"', '\"') + '"')
    line.append('"' + full_jobs[record]['posted'].replace('"', '\"') + '"')
    line.append('"' + full_jobs[record]['href'].replace('"', '\"') + '"')
    line.append('"' + '|'.join(full_jobs[record]['skills']).replace('"', '\"') + '"')
    line.append('"' + '|'.join(full_jobs[record]['industry']).replace('"', '\"').replace('&amp;', '&') + '"')
    line.append('"' + full_jobs[record]['description'].replace('"', '\"').replace("\n",'') + '"')
    lines.append(','.join(line))
output = "\n".join(lines)
print(output)
import csv
f = open('output.csv',"w")
f.write(output)
f.close()
    

In [ ]:
print(len(lines))